In [1]:
#GPU runtime required, should give CUDA version
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [2]:
!pip install faiss-gpu

     |████████████████████████████████| 85.5 MB 84 kB/s 


In [3]:
import faiss
import torch
import cv2
from google.colab.patches import cv2_imshow

In [4]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!cp -r /content/gdrive/MyDrive/Explainable_Wound_Classification/CMSF/self_supervised/* /content

In [6]:
!python train_msf_km.py \
  --cos \
  --weak_strong \
  --learning_rate 0.05 \
  --batch_size 32 \
  --num_workers 2 \
  --epochs 200 \
  --print_freq 10 \
  --arch resnet50 \
  --topk 5 \
  --momentum 0.99 \
  --mem_bank_size 128000 \
  --num_clusters 4 \
  --checkpoint_path /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output \
  /content/gdrive/MyDrive/Explainable_Wound_Classification/Split_Labeled_images/

/content/train_msf_km.py
import builtins
import os
import sys
import time
import argparse
import random

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torchvision import transforms, datasets

from PIL import ImageFilter
from util import adjust_learning_rate, AverageMeter, subset_classes
import models.resnet as resnet
from tools import get_logger

import pdb
import faiss


def parse_option():

    parser = argparse.ArgumentParser('argument for training')

    parser.add_argument('data', type=str, help='path to dataset')
    parser.add_argument('--dataset', type=str, default='imagenet',
                        choices=['imagenet', 'imagenet100'],
                        help='use full or subset of the dataset')
    parser.add_argument('--debug', action='store_true', help='whether in debug mode or not')

    parser.add_argument('--print_freq', type=int, default=100, help='print frequency')
    parser.add_argument('--save_freq', type=int, default=10, help='sa

In [7]:
!python eval_linear.py \
  --workers 2 \
  --arch resnet50 \
  --batch-size 32 \
  --save /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output_eval \
  --weights /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output/ckpt_epoch_200.pth \
  /content/gdrive/MyDrive/Explainable_Wound_Classification/Split_Labeled_images/

/content/eval_linear.py
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn.functional as F

from tools import *
#from models.alexnet import AlexNet
#from models.mobilenet import MobileNetV2
import models.resnet as resnet


parser = argparse.ArgumentParser(description='Unsupervised distillation')
parser.add_argument('data', metavar='DIR',
                    help='path to dataset')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('-a', '--arch', default='resnet18',
                    help='model architecture: ' +
                 

In [8]:
best_model = torch.load('/content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output_eval/model_best.pth.tar')
best_model

{'best_acc1': tensor(69.5652, device='cuda:0'),
 'epoch': 19,
 'lr_scheduler': {'_get_lr_called_within_step': False,
  '_last_lr': [0.001],
  '_step_count': 20,
  'base_lrs': [0.01],
  'gamma': 0.1,
  'last_epoch': 19,
  'milestones': Counter({15: 1, 30: 1, 40: 1}),
  'verbose': False},
 'optimizer': {'param_groups': [{'dampening': 0,
    'initial_lr': 0.01,
    'lr': 0.001,
    'momentum': 0.9,
    'nesterov': False,
    'params': [0, 1],
    'weight_decay': 0.0001}],
  'state': {0: {'momentum_buffer': tensor([[ 0.0014,  0.3129, -0.1872,  ...,  0.5888,  0.8816,  1.1801],
            [-0.0050, -0.5733,  0.1858,  ..., -0.4423, -0.5145, -0.9843],
            [ 0.0339,  0.2576, -0.1483,  ..., -0.1294, -0.5596, -0.3135],
            [-0.0304,  0.0027,  0.1497,  ..., -0.0171,  0.1925,  0.1176]],
           device='cuda:0')},
   1: {'momentum_buffer': tensor([ 0.8012, -0.6098,  0.1089, -0.3003], device='cuda:0')}}},
 'state_dict': OrderedDict([('1.inv_std',
               tensor([ 54.6979, 1

In [12]:
!python eval_linear.py \
  --workers 2 \
  --arch resnet50 \
  --batch-size 32 \
  --evaluate \
  --resume /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output_eval/model_best.pth.tar \
  --save /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output_eval \
  --weights /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output/ckpt_epoch_200.pth \
  /content/gdrive/MyDrive/Explainable_Wound_Classification/Split_Labeled_images/

/content/eval_linear.py
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn.functional as F

from tools import *
#from models.alexnet import AlexNet
#from models.mobilenet import MobileNetV2
import models.resnet as resnet


parser = argparse.ArgumentParser(description='Unsupervised distillation')
parser.add_argument('data', metavar='DIR',
                    help='path to dataset')
parser.add_argument('-j', '--workers', default=4, type=int, metavar='N',
                    help='number of data loading workers (default: 4)')
parser.add_argument('-a', '--arch', default='resnet18',
                    help='model architecture: ' +
                 

In [13]:
!python eval_knn.py \
  --workers 2 \
  --arch resnet50 \
  --batch-size 32 \
  --save /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output_knn_eval \
  --weights /content/gdrive/MyDrive/Explainable_Wound_Classification/outputs/cmsf-km_4_cluster_output/ckpt_epoch_200.pth \
  /content/gdrive/MyDrive/Explainable_Wound_Classification/Split_Labeled_images/

/content/eval_knn.py
import builtins
from collections import Counter, OrderedDict
from random import shuffle
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn.functional as F
import numpy as np
import faiss

from tools import *
from models.resnet import resnet18, resnet50
#from models.alexnet import AlexNet as alexnet
#from models.mobilenet import MobileNetV2 as mobilenet
# from models.resnet_swav import resnet50w5, resnet50 as swav_resnet50
# from models.resnet_byol import resnet50 as byol_resnet50
# from models.resnet_gn_ws import l_resnet18, l_resnet50
from eval_linear import load_weights
# from file_dataset import FileDataset


parser = ar